In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
max_long = 25

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

In [6]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [7]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [8]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
df = data.get_data()
cleaned_df = data.clean_data(df)

In [14]:
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, token)

In [15]:
word_vector_matrix = token_glove.glove_vector(token)

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [18]:
y = cleaned_df['target']

In [39]:
model = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(LSTM(128,return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(32, return_sequences = False))
model.add(Dropout(0.4))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=['accuracy'])
history = model.fit(X, y, batch_size=16, epochs = 7, validation_split=0.1, callbacks = [es])

Epoch 1/7
429/429 [==============================] - 40s 84ms/step - loss: 0.5655 - accuracy: 0.7383 - val_loss: 0.4300 - val_accuracy: 0.8215
Epoch 2/7
429/429 [==============================] - 33s 76ms/step - loss: 0.4508 - accuracy: 0.8137 - val_loss: 0.4333 - val_accuracy: 0.8097
Epoch 3/7
429/429 [==============================] - 31s 72ms/step - loss: 0.4128 - accuracy: 0.8333 - val_loss: 0.4208 - val_accuracy: 0.8123
Epoch 4/7
429/429 [==============================] - 34s 78ms/step - loss: 0.3941 - accuracy: 0.8416 - val_loss: 0.4289 - val_accuracy: 0.8123
Epoch 5/7
429/429 [==============================] - 34s 78ms/step - loss: 0.3732 - accuracy: 0.8520 - val_loss: 0.4346 - val_accuracy: 0.8031
Epoch 6/7
429/429 [==============================] - 33s 78ms/step - loss: 0.3483 - accuracy: 0.8641 - val_loss: 0.4699 - val_accuracy: 0.8045
Epoch 7/7
429/429 [==============================] - 34s 79ms/step - loss: 0.3325 - accuracy: 0.8701 - val_loss: 0.4739 - val_accuracy: 0.7940

#Phase de Test

In [11]:
df = data.get_data(test=True)
test_df = data.clean_data(df)

In [12]:
text_test = token_glove.create_list(test_df['text'])
token_test = token_glove.token_ize(text_test)
X_test = token_glove.token_tweet(text_test, token_test)

In [40]:
results = model.predict(X_test)

In [41]:
results

array([[0.5938771 ],
       [0.74081326],
       [0.9817258 ],
       ...,
       [0.29881954],
       [0.10856587],
       [0.31490782]], dtype=float32)

In [42]:
def arrondi(n):
  if n >= 0.55:
    return 1
  else :
    return 0

In [43]:
to_submit = [arrondi(x[0]) for x in results]

In [44]:
to_submit_df = pd.DataFrame(to_submit)

In [45]:
to_submit_df.rename(columns={0: 'target'}, inplace=True)

In [46]:
to_submit_df['id'] = test_df.id

In [47]:
to_submit_df = to_submit_df.set_index('id')

In [48]:
to_submit_df.to_csv('/content/gdrive/My Drive/submission1b.csv')

In [38]:
model.save('/content/gdrive/My Drive/RNN_camille_1CSV_v1')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_camille_1CSV_v1/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_camille_1CSV_v1/assets
